In [1]:
import tabula as tb
import pandas as pd
import PyPDF2
from PyPDF2 import PdfReader
import re
import docx
from docx import Document
import zipfile
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [2]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ', ' ,':',', '  ':' '
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = " ".join(str.split())
    str = str.strip('\n')
    str = str.strip(' ')
    return str

def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./result/2021.csv', index=False, encoding="utf-8")

In [97]:
### code entry
path = "./data/2021.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式
totaldata = handle(bs)
to_CSV(totaldata)

topic name: Focus Area 1 In-Space Propulsion Technologies
subtopic name: Z10.01Cryogenic Fluid Management
subtopic name: Z10.03Space Nuclear Propulsion
subtopic name: Z10.04Materials, Processes, and Technologies for Advancing In-Space Electric Propulsion Thrusters
topic name: Focus Area 2 Power, Energy and Storage
subtopic name: S3.01Power Generation and Conversion
subtopic name: S3.02Dynamic Power Conversion
subtopic name: S3.03Energy Storage for Extreme Environments
subtopic name: Z1.05Lunar and Planetary Surface Power Management and Distribution
subtopic name: Z1.06Radiation-Tolerant High-Voltage, High-Power Electronics
subtopic name: Z1.07Dynamic Energy Conversion for Space Nuclear Power and Propulsion
topic name: Focus Area 3 Autonomous Systems for Space Exploration
subtopic name: H6.22Deep Neural Net and Neuromorphic Processors for In-Space Autonomy and Cognition
subtopic name: H6.23Spacecraft Autonomous Agent Cognitive Architectures for Human Exploration
subtopic name: S5.05Faul

In [96]:
def handle(bs):
    totalData = []
    ### get overall topic
    evens = bs.find_all("li", {"class": "md-taxonomy even"})
    odds = bs.find_all("li", {"class": "md-taxonomy odd"})
    for index in range(24):
        dic = {
            "Focus Area": "",
            "Subtopic": "",
            "Lead Center": "",
            "Participating Center(s)": "",
            "Solicitation Year": "",
            "Scope Title": "",
            "Scope Description": "",
            "Expected TRL or TRL Range at completion of the Project": "",
            "Primary Technology Taxonomy": "",
            "Desired Deliverables of Phase I and Phase II": "",
            "Desired Deliverables Description": "",
            "State of the Art and Critical Gaps": "",
            "Relevance / Science Traceability": "",
            "References": ""
        }
        topic_block = evens[int(index / 2)] if int(index % 2) == 0 else odds[int(index / 2)]
        dic["Focus Area"] = getTopicName(topic_block)
        ### get sub-focus topics
        subtopic_even = topic_block.find_all("li", {"class": "first-topic even"})
        subtopic_odd = topic_block.find_all("li", {"class": "first-topic odd"})
        for index2 in range(len(subtopic_even) + len(subtopic_odd)):
            subtopic_block = subtopic_even[int(index2 / 2)] if int(index2 % 2) == 0 else subtopic_odd[int(index2 / 2)]
            dic["Subtopic"] = getSubtopicName(subtopic_block)
            metaDic = getMetaData(subtopic_block)
            for k in metaDic:
                if k in dic.keys():
                    dic[k] = metaDic[k]
            ### get other data
            whole_info_block = subtopic_block.find("div", {"class": "whole-topic-content"})
            scope_num = getScopeNum(whole_info_block)
            for i in range(scope_num):
                new_dic = dic.copy()
                new_dic["Scope Title"] = getScopeTitle(whole_info_block, i)
                new_dic["Scope Description"] = getScopeDescription(whole_info_block, i)
                metaDic2 = getTRLAndPTTAndDD(subtopic_block, i)
                for k in metaDic2:
                    if k in new_dic.keys():
                        new_dic[k] = metaDic2[k]
                new_dic["Desired Deliverables Description"] = getDesiredDeliverablesDescription(whole_info_block, i)
                new_dic["State of the Art and Critical Gaps"] = getGaps(whole_info_block, i)
                new_dic["Relevance / Science Traceability"] = getTraceability(whole_info_block, i)
                new_dic["References"] = getReference(whole_info_block, i)
                totalData.append(new_dic)
    return totalData

In [17]:
### get topic and subtopic name
def getTopicName(topic_block):
    topic_name = filterHTMLstr(topic_block.find("h2").get_text())
    print("topic name:", topic_name)  # topic name
    return topic_name

### handle every sub-topic
def getSubtopicName(subtopic_block):
    subtopic_name = filterHTMLstr(subtopic_block.find("p", {"class": "subtopic"}).get_text())
    print("subtopic name:", subtopic_name)  # subtopic name
    return subtopic_name

In [19]:
### get Lead Center, Participating Center(s), Solicitation Year
def getMetaData(subtopic_block):
    dic = {}
    info_group = subtopic_block.find_all("p", {"class": "l-center"})
    for raw_info in info_group:
        info = filterHTMLstr(raw_info.get_text())
        pair = info.split(":")
        dic[pair[0]] = pair[1]
        # print(pair[0], pair[1])
        # print(dic)
    return dic

In [66]:
### scope num
def getScopeNum(whole_info_block):
    scope_num = len(whole_info_block.find_all("h5"))
    # print("scope title:", scope_title)
    return scope_num

In [65]:
### scope title
def getScopeTitle(whole_info_block, target_scope_num):
    scope_title = filterHTMLstr(whole_info_block.find_all("h5")[target_scope_num].get_text())
    # print("scope title:", scope_title)
    return scope_title

In [68]:
### scope description
def getScopeDescription(whole_info_block, target_scope_num):
    scope_description = ""
    flag = False
    scope_num = 0
    for tag in whole_info_block.find_all("p"):
        # print(filterHTMLstr(tag.get_text()))
        if flag:
            break
        if tag.get_text() != "" and "Scope Description" in filterHTMLstr(tag.get_text()):
            # print(filterHTMLstr(tag.get_text()))
            for info in tag.next_siblings:
                if info.get_text() != "" and "Expected TRL or TRL Range at completion of the Project" in filterHTMLstr(info.get_text()) and scope_num == target_scope_num:
                    flag = True
                    break
                if scope_num != target_scope_num:
                    scope_num += 1
                    continue
                scope_description += filterHTMLstr(info.get_text())
                # print(filterHTMLstr(info.get_text()))
    return scope_description

In [95]:
### TRL, Primary Technology Taxonomy, Desired Deliverables of Phase I and Phase II
def getTRLAndPTTAndDD(whole_info_block, target_scope_num):
    trl = ""
    info_block = ""
    dic = {}
    scope_num = 0
    for info in whole_info_block.find_all("p"):
        if info.get_text() != "" and "Expected TRL or TRL Range at completion of the Project" in filterHTMLstr(info.get_text()):
            if scope_num != target_scope_num:
                scope_num += 1
                continue
            info_block = info
    
    infos = info_block.find_all("strong")
    for i in infos:
        key = filterHTMLstr(i.get_text())
        key = key.replace(":", "")
        data = ""
        for j in i.next_siblings:
            if j.name == "strong":
                if data.strip() == "":
                    continue
                # print("data", data)
                break
            if j.name == "br":
                data += '\n'
            else:
                data += filterHTMLstr(j.get_text())
        if key == "Desired Deliverables of Phase I and Phase II":
            deliverable_block = info_block.find_next_sibling("ul")
            for k in deliverable_block.find_all("li"):
                data += filterHTMLstr(k.get_text())
                data += '\n'
            # print(info_block)
        dic[key] = data.strip()
    return dic

In [59]:
### Desired Deliverables Description
def getDesiredDeliverablesDescription(whole_info_block, target_scope_num):
    desired_deliverable_description = ""
    flag = False
    scope_num = 0
    for tag in whole_info_block.find_all("p"):
        # print(filterHTMLstr(tag.get_text()))
        if flag:
            break
        if tag.get_text() != "" and "Desired Deliverables Description" in filterHTMLstr(tag.get_text()):
            # print(filterHTMLstr(tag.get_text()))
            if scope_num != target_scope_num:
                scope_num += 1
                continue
            for info in tag.next_siblings:
                if info.get_text() != "" and "State of the Art and Critical Gaps" in filterHTMLstr(info.get_text()) and scope_num == target_scope_num:
                    flag = True
                    break
                desired_deliverable_description += filterHTMLstr(info.get_text())
                # print(filterHTMLstr(info.get_text()))
    return desired_deliverable_description

In [60]:
### State of the Art and Critical Gaps
def getGaps(whole_info_block, target_scope_num):
    state_of_the_art_critical_gaps = ""
    flag = False
    scope_num = 0
    for tag in whole_info_block.find_all("p"):
        # print(filterHTMLstr(tag.get_text()))
        if flag:
            break
        if tag.get_text() != "" and "State of the Art and Critical Gaps" in filterHTMLstr(tag.get_text()):
            # print(filterHTMLstr(tag.get_text()))
            if scope_num != target_scope_num:
                scope_num += 1
                continue
            for info in tag.next_siblings:
                if info.get_text() != "" and "Relevance / Science Traceability" in filterHTMLstr(info.get_text()) and scope_num == target_scope_num:
                    flag = True
                    break
                state_of_the_art_critical_gaps += filterHTMLstr(info.get_text())
                # print(filterHTMLstr(info.get_text()))
    return state_of_the_art_critical_gaps

In [61]:
### Relevance / Science Traceability
def getTraceability(whole_info_block, target_scope_num):
    traceability = ""
    flag = False
    scope_num = 0
    for tag in whole_info_block.find_all("p"):
        # print(filterHTMLstr(tag.get_text()))
        if flag:
            break
        if tag.get_text() != "" and "Relevance / Science Traceability" in filterHTMLstr(tag.get_text()):
            # print(filterHTMLstr(tag.get_text()))
            if scope_num != target_scope_num:
                scope_num += 1
                continue
            for info in tag.next_siblings:
                if info.get_text() != "" and "References" in filterHTMLstr(info.get_text()) and scope_num == target_scope_num:
                    flag = True
                    break
                traceability += filterHTMLstr(info.get_text())
                # print(filterHTMLstr(info.get_text()))
    return traceability

In [62]:
### References
def getReference(whole_info_block, target_scope_num):
    reference = ""
    flag = False
    scope_num = 0
    for tag in whole_info_block.find_all("p"):
        if tag.get_text() != "" and "References" in filterHTMLstr(tag.get_text()):
            if scope_num != target_scope_num:
                scope_num += 1
                continue
            for content in tag.next_siblings:
                item = filterHTMLstr(content.get_text())
                if "Read less" in item or "Scope Title" in item:
                    break
                # print(content.name)
                # print(item)
                # reference_block = tag.find_next_sibling("ol")
                if content.name == "ol" or content.name == "ul":
                    i = 1
                    for k in content.find_all("li"):
                        reference += str(i) + ". " + filterHTMLstr(k.get_text())
                        reference += '\n'
                        i += 1
                else:
                    reference += item
                    reference += '\n'
                    # print(filterHTMLstr(info.get_text()))
    # print(reference)
    return reference